## Kütüphanelerin eklenmesi ve Google Drive'a erişim için yetki verilmesi.
### CVS dosyasının okunması ve df kısaltmasına atanması.
- #### Dataset dosyasının konumu drive/MyDrive/Colab Notebooks/TDDI2022 olmalıdır.
- #### Dataset dosyasının adı "kanunum-nlp-doc-analysis-dataset.csv" olmalıdır.

In [8]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/

In [ ]:
%cd MyDrive/

In [ ]:
%cd Colab Notebooks/

In [ ]:
# Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 diye bir klasör oluşturduktan sonra o klasöre erişiyoruz.
# Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

## İlgili alanların üstmetinlerinin görüntülenebilmesini sağlayan kodlar.

In [ ]:
print(df.groupby('kategori').size())

In [ ]:
print(df.loc[df['kategori'] == 'Kanun Hükmünde Kararname'])

In [ ]:
print(df['data_text'][850][0:265])

## Transformers modülünün 4.20.1 sürümünde yüklenmesi

In [ ]:
!pip install transformers==4.20.1

## Elde edilen verilerin datasetteki doğru verilerle karşılaştıran fonksiyon.

In [24]:
def kontrol(ry,ra,rg,rsayi,mno,my,ma,mg,liste,satir):
  if int(mg) < 10:
    mg = ('0'+mg)

  if int(ma) < 10:
    ma = ('0'+ma)

  if df['rega_tarihi'][satir] == (ry+'-'+ra+'-'+rg):
    liste.append('1')
  else:
    liste.append('0')

  if df['rega_no'][satir] ==rsayi:
    liste.append('1')
  else:
    liste.append('0')

  if df['mevzuat_no'][satir] == mno:
    liste.append('1')
  else:
    liste.append('0')

  if df['mevzuat_tarihi'][satir] == (my+'-'+ma+'-'+mg):
    liste.append('1')
  else:
    liste.append('0')

  return liste

## TDDI kullanılarak "data_text"ten verilerin alınması.
- #### Kullanılan Model: "lserinol/bert-turkish-question-answering"
- #### Kullanılan NLP görevi: question-answering
- #### Çekilen Değerler: 
 1.   data_text
 2.   rega_no
 3.   rega_tarihi
 4.   mevzuat_no
 5.  mevzuat_tarihi

- #### Bulunan değerlerin gerçek değerlerle karşılaştırılması ve doğruluk oranının bulunması






In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

modeller = "lserinol/bert-turkish-question-answering"

for satir in range(826,917):
  total = 0
  dogru = 0
  icerik = df['data_text'][satir][:123]
  if icerik.find("Mükerrer") == -1:
    icerik = df['data_text'][satir][:116]
  else:
    icerik = df['data_text'][satir][:123]
  
  print(icerik)
  sorular = ["Resmi Gazete Tarihi nedir?","Resmi Gazete Sayısı nedir?","Karar Sayısı Nedir?","Kararnamenin Tarihi nedir?"]

  cevaplar = []
  veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(modeller), tokenizer=AutoTokenizer.from_pretrained(modeller))
  for soru in sorular:
      sonuc = veriBulucu(question = soru, context = icerik)
      cevaplar.append(sonuc['answer'])
      #print("'",soru,"'","sorusuna verilen cevap:",sonuc['answer'])
  try:
    rg ,ra, ry = str(cevaplar[0]).split('.')
  except:
    rg ,ra, ry = str(cevaplar[0]).split('/')

  rsayi = cevaplar[1]
  khk , mno = cevaplar[2].split('/')
  try:
    mg , ma, my = str(cevaplar[3]).split('/')
  except:
    mg , ma, my = str(cevaplar[3]).split('.')
  
  liste = []
  cıktı = kontrol(ry,ra,rg,rsayi,mno,my,ma,mg,liste,satir)
  if cıktı == ['1','1','1','1']:
    dogru+=1
  total+=1
  print(satir,cıktı,cevaplar)
print("Toplam",total,"girdi,",dogru,"dogru girdi")